Courcera Capstone

This notebook will be used for IBM Data science noteboook


In [1]:
import pandas as pd 
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
import urllib.request

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
page = urllib.request.urlopen(url)

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(page, "lxml")

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"449ccf20-e1f6-4d16-813e-c13126a76ab2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":969510799,"wgRevisionId":969510799,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [9]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [10]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        

In [11]:
df=pd.DataFrame(A,columns=['Postal codes'])
df['Borough']=B
df['Neighbourhood']=C
df

,Postal codes,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [12]:
df.shape

(180, 3)

In [13]:
df["Borough"].value_counts()

Not assigned
        77
North York
          24
Downtown Toronto
    19
Scarborough
         17
Etobicoke
           12
Central Toronto
      9
West Toronto
         6
East Toronto
         5
East York
            5
York
                 5
Mississauga
          1
Name: Borough, dtype: int64

In [14]:
df.head()

,Postal codes,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
df1=df[~df.Borough.str.contains("Not assigned")]

In [16]:
df1.head()

,Postal codes,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
df1['Borough'].value_counts()

North York
          24
Downtown Toronto
    19
Scarborough
         17
Etobicoke
           12
Central Toronto
      9
West Toronto
         6
East Toronto
         5
East York
            5
York
                 5
Mississauga
          1
Name: Borough, dtype: int64

In [18]:
df1

,Postal codes,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [19]:
df1.shape

(103, 3)

In [22]:
!wget -O Geo.csv http://cocl.us/Geospatial_data

--2020-08-01 05:47:20--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.21, 159.8.69.24, 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.69.21|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-08-01 05:47:20--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.69.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-01 05:47:24--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-01 05:47:24--  https://ibm.box.com/public/static/9a

In [23]:
df2 = pd.read_csv("Geo.csv", delimiter=",")
df2[0:5]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
df2['Postal Code'].str.strip()

0      M1B
1      M1C
2      M1E
3      M1G
4      M1H
5      M1J
6      M1K
7      M1L
8      M1M
9      M1N
10     M1P
11     M1R
12     M1S
13     M1T
14     M1V
15     M1W
16     M1X
17     M2H
18     M2J
19     M2K
20     M2L
21     M2M
22     M2N
23     M2P
24     M2R
25     M3A
26     M3B
27     M3C
28     M3H
29     M3J
      ... 
73     M6C
74     M6E
75     M6G
76     M6H
77     M6J
78     M6K
79     M6L
80     M6M
81     M6N
82     M6P
83     M6R
84     M6S
85     M7A
86     M7R
87     M7Y
88     M8V
89     M8W
90     M8X
91     M8Y
92     M8Z
93     M9A
94     M9B
95     M9C
96     M9L
97     M9M
98     M9N
99     M9P
100    M9R
101    M9V
102    M9W
Name: Postal Code, Length: 103, dtype: object

In [28]:
df2.rename(columns={'Postal Code':'Postal codes'}, inplace=True)

In [29]:
df2

,Postal codes,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [31]:
df1.reset_index(drop=True, inplace=True)

In [38]:
df1= df1.groupby(["Postal codes", "Borough"], as_index = False).agg(",".join)

In [42]:
df3=df1.merge(df2, on='Postal codes',how='outer')

In [43]:
df1

,Postal codes,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge\n",NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n",NaN,NaN
3,M1G,Scarborough,Woburn\n,NaN,NaN
4,M1H,Scarborough,Cedarbrae\n,NaN,NaN


,Postal codes,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [86]:
df1['Postal codes'].str.strip() 

2      M3A
3      M4A
4      M5A
5      M6A
6      M7A
8      M9A
9      M1B
11     M3B
12     M4B
13     M5B
14     M6B
17     M9B
18     M1C
20     M3C
21     M4C
22     M5C
23     M6C
26     M9C
27     M1E
30     M4E
31     M5E
32     M6E
36     M1G
39     M4G
40     M5G
41     M6G
45     M1H
46     M2H
47     M3H
48     M4H
      ... 
111    M4R
112    M5R
113    M6R
114    M7R
116    M9R
117    M1S
120    M4S
121    M5S
122    M6S
126    M1T
129    M4T
130    M5T
135    M1V
138    M4V
139    M5V
142    M8V
143    M9V
144    M1W
147    M4W
148    M5W
151    M8W
152    M9W
153    M1X
156    M4X
157    M5X
160    M8X
165    M4Y
168    M7Y
169    M8Y
178    M8Z
Name: Postal codes, Length: 103, dtype: object

In [87]:
df2['Postal codes'].str.strip() 

0      M1B
1      M1C
2      M1E
3      M1G
4      M1H
5      M1J
6      M1K
7      M1L
8      M1M
9      M1N
10     M1P
11     M1R
12     M1S
13     M1T
14     M1V
15     M1W
16     M1X
17     M2H
18     M2J
19     M2K
20     M2L
21     M2M
22     M2N
23     M2P
24     M2R
25     M3A
26     M3B
27     M3C
28     M3H
29     M3J
      ... 
73     M6C
74     M6E
75     M6G
76     M6H
77     M6J
78     M6K
79     M6L
80     M6M
81     M6N
82     M6P
83     M6R
84     M6S
85     M7A
86     M7R
87     M7Y
88     M8V
89     M8W
90     M8X
91     M8Y
92     M8Z
93     M9A
94     M9B
95     M9C
96     M9L
97     M9M
98     M9N
99     M9P
100    M9R
101    M9V
102    M9W
Name: Postal codes, Length: 103, dtype: object

In [92]:
 df3=df1.groupby(['Postal codes','Borough'], sort = False).agg(lambda x: ','.join(x))

In [94]:
df3

,,Neighbourhood
Postal codes,Borough,
M3A,North York,Parkwoods\n
M4A,North York,Victoria Village\n
M5A,Downtown Toronto,"Regent Park, Harbourfront\n"
M6A,North York,"Lawrence Manor, Lawrence Heights\n"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government\n"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village\n"
M1B,Scarborough,"Malvern, Rouge\n"
M3B,North York,Don Mills\n
M4B,East York,"Parkview Hill, Woodbine Gardens\n"


In [109]:
df1.dtypes

Postal codes     object
Borough          object
Neighbourhood    object
dtype: object

In [110]:
df2.dtypes

Postal codes     object
Latitude        float64
Longitude       float64
dtype: object

In [131]:
df5=pd.merge(df1,df2,left_on='Postal codes',right_on='Postal codes',left_index=False,right_index=False)

In [132]:
df5

,Postal codes,Borough,Neighbourhood,Latitude,Longitude
